In [35]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [36]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [70]:
def ode_model(z, t, beta, tao, zeta, eta, delta, epsilon, kappa, phi, theta1, theta2, alpha2, alpha1, sigma): #Added all parameters Reordered 
    S, L, E, Ir, Iu, Q, R, D = z #Added Is, Ia Q D terms for reported and unreported cases
    N = S + L + E + Ir + Iu + Q + R + D #Added Ir, Iu, Q, D to the sum
    dSdt = -beta*S*(Ir + tao*Iu)/N - eta*S + delta*L + zeta*E #Added I = Ir + Iu and tao*Iu is the reduced transmission rate; zeta*E is the recovery rate from E to S; eta and delta are related to the lockdown
    dLdt = eta*S -delta*L ##
    dEdt = beta*S*(Ir + tao*Iu)/N - (epsilon + zeta)*E #Added I and (sigma + zeta)
    dIrdt = epsilon*kappa*E - phi*Iu - theta1*Ir 
    dIudt = epsilon*(1 - kappa)*E - (theta2 + phi)*Iu # Iu unreported
    dQdt = theta1*Ir + theta2*Iu - alpha1*Q
    dRdt = alpha2*Iu + alpha1*Q - sigma*R
    dDdt = sigma*R
    
    return [dSdt, dLdt, dEdt, dIrdt, dIudt, dQdt, dRdt, dDdt] #S, L, E, R, U, Q, R, D

In [71]:
def ode_solver(t, initial_conditions, params):
    initE, initL, initIr, initIu, initQ, initR, initD, initN= initial_conditions #Added all init names
    beta, tao, zeta, eta, delta, epsilon, kappa, phi, theta1, theta2, alpha2, alpha1, sigma = params #Added all parameters
    initS = initN - (initL + initE + initIr + initIu + initQ + initR + initD)
    res = odeint(ode_model, [initS, initL, initE, initIr, initIu, initQ, initR, initD], t, args=(beta, tao, zeta, eta, delta, epsilon, kappa, phi, theta1, theta2, alpha2, alpha1, sigma))
    return res

In [72]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
#NOTE: those initial conditions do not represent any physical meaning, test number only
initN = 2930000
initL = 0
initE = 1
initIr = 1
initIu = 1
initQ = 0
initR = 0
initD = 0
beta = 2
tao = 0.2
zeta = 0.05
eta = 0.04
delta = 0.01
epsilon = 0.95
kappa = 0.8
phi = 0.7
theta1 = 0.2
theta2 = 0.13
alpha2 = 0.12
alpha1 = 0.2
sigma = 0.05
days = 200

In [73]:
def main(initL, initE, initIr, initIu, initQ, initR, initD, initN, beta, tao, zeta, eta, delta, epsilon, kappa, phi, theta1, theta2, alpha2, alpha1, sigma, days):
    initial_conditions = [initL, initE, initIr, initIu, initQ, initR, initD, initN]
    params = [beta, tao, zeta, eta, delta, epsilon, kappa, phi, theta1, theta2, alpha2, alpha1, sigma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, L, E, Ir, Iu, Q, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=L, mode='lines+markers', name='Lockdown'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=Ir, mode='lines+markers', name='Reported Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=Iu, mode='lines+markers', name='Unreported Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=Q, mode='lines+markers', name='Quarantine'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))
    fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines+markers', name='Deceased'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of SLERUQRD Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    #fig.write_image("images/seird_simulation.png")
    fig.show()

In [74]:
interact(main, 
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, layout=slider_layout),
         initL=IntSlider(min=0, max=100000, step=1, value=initL, description='initL', style=style, layout=slider_layout),
         initIr=IntSlider(min=0, max=100000, step=10, value=initIr, description='initIr', style=style, layout=slider_layout),
         initIu=IntSlider(min=0, max=100000, step=10, value=initIu, description='initIu', style=style, layout=slider_layout),
         initQ=IntSlider(min=0, max=100000, step=10, value=initQ, description='initQ', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
         initD=IntSlider(min=0, max=100000, step=10, value=initD, description='initD', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
         tao=FloatSlider(min=0, max=1, step=0.01, value=tao, description='Unreported Reduced/Increased Infection Rate', style=style, layout=slider_layout),
         zeta=FloatSlider(min=0, max=1, step=0.01, value=zeta, description='Recovery from Exposure rate', style=style, layout=slider_layout),
         eta=FloatSlider(min=0, max=1, step=0.01, value=eta, description='Lockdown Rate', style=style, layout=slider_layout),
         delta=FloatSlider(min=0, max=1, step=0.01, value=delta, description='Lockdown Exit Rate', style=style, layout=slider_layout),
         epsilon=FloatSlider(min=0, max=1, step=0.01, value=epsilon, description='Exposure to Infection rate', style=style, layout=slider_layout), 
         kappa=FloatSlider(min=0, max=1, step=0.01, value=kappa, description='% Reported Infection', style=style, layout=slider_layout),
         phi=FloatSlider(min=0, max=1, step=0.01, value=phi, description='Unreported to Reported rate', style=style, layout=slider_layout),
         theta1=FloatSlider(min=0, max=1, step=0.01, value=theta1, description='Reported to Quarantine rate', style=style, layout=slider_layout),
         theta2=FloatSlider(min=0, max=1, step=0.01, value=theta2, description='Unreported to Quarantine rate', style=style, layout=slider_layout),
         alpha2=FloatSlider(min=0, max=1, step=0.01, value=alpha2, description='Unreported Recovery rate', style=style, layout=slider_layout),
         alpha1=FloatSlider(min=0, max=1, step=0.01, value=alpha1, description='Quarantine Recovery rate', style=style, layout=slider_layout),
         sigma=FloatSlider(min=0, max=1, step=0.01, value=sigma, description='Death rate', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        );

interactive(children=(IntSlider(value=0, description='initL', layout=Layout(width='99%'), max=100000, style=Sl…